# ゼロから作る Deep Learning 3 フレームワーク編

## ステップ1 箱としての変数

In [1]:
# 変数用のクラス
class Variable:
  def __init__(self, data):
    self.data = data

In [2]:
import numpy as np

data = np.array(1.0)
x =Variable(data)
print(x.data)

1.0
